<h1>Coursera Capstone Week 3 Part 3</h1>

In this project, I am going to scrap the table from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, and organize it into a pandas dataframe, and clean the data.
In Part 2, I am going to create another dataframe of the latitudes and longitudes from a csv file from http://cocl.us/Geospatial_data and merge the 2 dataframes with appropriate postalcode. In Part 3, I am going to explore and cluster the neighborhoods in the dataframe.

<h3>Part 1 </h3>

<h2> Scraping the Data</h2>

In [1]:
import pandas as pd


In [2]:
from bs4 import BeautifulSoup
import requests

Use the BeautifulSoup library and the requests library to scrape html text

In [3]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Soup should return the html code

In [4]:
soup = BeautifulSoup(source, 'html.parser')



Data should return the html code enclosed by the class as noted

In [5]:
data = soup.find('table',{'class':'wikitable sortable'})

The read_html function returns a raw string from data

In [6]:
df = pd.read_html(str(data))

In [7]:
df

[            0                 1  \
 0    Postcode           Borough   
 1         M1A      Not assigned   
 2         M2A      Not assigned   
 3         M3A        North York   
 4         M4A        North York   
 5         M5A  Downtown Toronto   
 6         M5A  Downtown Toronto   
 7         M6A        North York   
 8         M6A        North York   
 9         M7A      Queen's Park   
 10        M8A      Not assigned   
 11        M9A         Etobicoke   
 12        M1B       Scarborough   
 13        M1B       Scarborough   
 14        M2B      Not assigned   
 15        M3B        North York   
 16        M4B         East York   
 17        M4B         East York   
 18        M5B  Downtown Toronto   
 19        M5B  Downtown Toronto   
 20        M6B        North York   
 21        M7B      Not assigned   
 22        M8B      Not assigned   
 23        M9B         Etobicoke   
 24        M9B         Etobicoke   
 25        M9B         Etobicoke   
 26        M9B         Etobi

The DataFrame function returns df as a pandas dataframe

In [8]:
df = pd.DataFrame(df[0])
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


<h2>Data Cleansing</h2>

Create a list of the header names

In [9]:
names = ['Postalcode','Borough','Neighborhood']

Replace the header with the list from above

In [10]:
df = df[1:] #take the data less the header row
df.columns = names #set the header row as the df header

Drops rows where the borough isn't specified

In [11]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.head()

,Postalcode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Loop through the table. If the neighborhood isn't assigned, assign it with the respective borough

In [12]:
for x in range (len(df.index)):
    if df.iloc[x][2] == 'Not assigned':
        df.iloc[x][2] = df.iloc[x][1]

Loop through the table. Neighborhoods with the same postal code should be seperated with a comma under the same row

In [14]:
for x in range (len(df.index)):
    if x < len(df.index)-1:
        check = True
        while check == True:
            if df.iloc[x][0] == df.iloc[x+1][0]:
                df.iloc[x][2] ="{0}, {1}".format(df.iloc[x][2], df.iloc[x+1][2]) 
                df = df.drop(df.index[x+1])
            else:
                check = False

In [15]:
df.shape

(103, 3)

Reset indices 

In [16]:
df = df.reset_index(drop=True)

In [17]:
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Construct a dataframe of latitude, longitude, and postalcode from the csv file below

<h3>Part 2</h3>

In [18]:
dfLatLong = pd.read_csv('http://cocl.us/Geospatial_data')

Rename the column on this dataframe so it maches that on the first dataframe

In [19]:
dfLatLong = dfLatLong.rename(columns={'Postal Code': 'Postalcode'})

In [20]:
dfLatLong.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two dataframe with appropriate postalcode

In [21]:
mergeDf = pd.merge(df, dfLatLong[['Postalcode','Latitude','Longitude']], on = 'Postalcode')

In [22]:
mergeDf.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


<h3>Part 3</h3>

Import Necessary Libraries


In [25]:
import numpy as np # library to handle data in a vectorized manner


import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


folium-0.5.0         | 45 KB     | #############################

Use the geolocator to find the latitude and longitude of Toronto

<h2>Creating a map of Toronto</h2>

In [29]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.653963 -79.387207


Create a map of Toronto using folium and the merged dataframe

In [33]:
map_toron = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(mergeDf['Latitude'], mergeDf['Longitude'], mergeDf['Borough'], mergeDf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toron)  
    
map_toron